This script is designed to predict stock prices using a Long Short-Term Memory neural network. 

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

Data is in the form of a CSV file

In [14]:
# Load the dataset
data_path = '/Users/dylanneal/Documents/mlops-stock-price-prediction/notebooks/stock_data/ABBV.csv'
abbv_data = pd.read_csv(data_path)

The 'date' column is converted to datetime format adn set as the DataFrame index 

In [15]:
# Convert the 'date' column to datetime format and set as index
abbv_data['date'] = pd.to_datetime(abbv_data['date'])
abbv_data.set_index('date', inplace=True)

Feature Selection for 'open, 'high', 'low', and 'close' prices for model input

In [16]:
# Select features to be used in the model
features = abbv_data[['open', 'high', 'low', 'close']].values

Feature scaling between 0 and 1 to improve neural network performance

In [17]:
# Scale the features to be between 0 and 1
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(features)

Look back defines how many previous timesteps are used to predict the next time step

In [18]:
# Define look_back period
look_back = 60

In [19]:
# Function to create dataset with multiple features
def create_dataset(dataset, look_back=60):
    X, Y = [], []
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i:(i + look_back), :]
        X.append(a)
        Y.append(dataset[i + look_back, -1])  # Target is still 'close' price
    return np.array(X), np.array(Y)

In [20]:
# Create dataset
X, y = create_dataset(data_scaled)

The dataset is split into training and testing parts based on a percentage of 67% 

In [21]:
# Splitting data into train and test sets
train_size = int(len(X) * 0.67)
test_size = len(X) - train_size
trainX, testX = X[:train_size], X[train_size:]
trainY, testY = y[:train_size], y[train_size:]

Model Architecture: 
    The model consists of two LSTM layers interspersed with 
        Dropout layers to prevent overfitting, 
        and a Dense layer for output. 
The model is compiled with the mean squared error loss function and the adam optimizer. 

In [22]:
# Create and compile the LSTM model
model = Sequential([
    Input(shape=(trainX.shape[1], 4)),
    LSTM(50, return_sequences=True),
    Dropout(0.2),
    LSTM(50),
    Dropout(0.2),
    Dense(1)
])
adam_optimizer = Adam(learning_rate=0.0001)
model.compile(loss='mean_squared_error', optimizer=adam_optimizer)

Early Stopping is used to halt training when the validation loss hasn't improved for a specified number of epochs. 
This helps prevent overfitting. 

In [23]:
# Setup early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

LET'S TRAIN 

The model is trained using the training data with validation split to monitor performance during training.

In [24]:
# Fit the model
history = model.fit(trainX, trainY, validation_split=0.2, epochs=100, batch_size=10, verbose=2, callbacks=[early_stopping])

Epoch 1/100
149/149 - 3s - 22ms/step - loss: 0.0037 - val_loss: 0.0011
Epoch 2/100
149/149 - 2s - 15ms/step - loss: 0.0014 - val_loss: 7.2984e-04
Epoch 3/100
149/149 - 2s - 15ms/step - loss: 0.0011 - val_loss: 6.7537e-04
Epoch 4/100
149/149 - 2s - 15ms/step - loss: 0.0012 - val_loss: 7.9762e-04
Epoch 5/100
149/149 - 2s - 15ms/step - loss: 9.8460e-04 - val_loss: 8.6991e-04
Epoch 6/100
149/149 - 2s - 15ms/step - loss: 0.0010 - val_loss: 7.9509e-04
Epoch 7/100
149/149 - 2s - 15ms/step - loss: 9.0496e-04 - val_loss: 5.9923e-04
Epoch 8/100
149/149 - 2s - 15ms/step - loss: 9.0879e-04 - val_loss: 5.9439e-04
Epoch 9/100
149/149 - 2s - 15ms/step - loss: 9.3265e-04 - val_loss: 7.0030e-04
Epoch 10/100
149/149 - 2s - 15ms/step - loss: 8.1523e-04 - val_loss: 6.3040e-04
Epoch 11/100
149/149 - 2s - 15ms/step - loss: 7.8729e-04 - val_loss: 7.7500e-04
Epoch 12/100
149/149 - 2s - 15ms/step - loss: 8.1397e-04 - val_loss: 5.3989e-04
Epoch 13/100
149/149 - 2s - 16ms/step - loss: 8.3855e-04 - val_loss: 8.44

Predictions are made for both training and testing datasets

In [25]:
# Making predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


Since the output was scaled, the predictions are rescaled back to the original scale to interpret them in the context of actual stock prices. 

In [ ]:
# Inverting predictions to revert back to the original scale
trainPredict = scaler.inverse_transform(np.c_[trainPredict, np.zeros((len(trainPredict), 3))])[:, 0]
testPredict = scaler.inverse_transform(np.c_[testPredict, np.zeros((len(testPredict), 3))])[:, 0]
trainY_original = scaler.inverse_transform(np.c_[trainY, np.zeros((len(trainY), 3))])[:, 0]
testY_original = scaler.inverse_transform(np.c_[testY, np.zeros((len(testY), 3))])[:, 0]


The root Mean Squared Error (RMSE) is calculated for both trainig and testing predictions to evaluate the performance of the model. 

In [ ]:
# Calculate root mean squared error
trainScore = np.sqrt(mean_squared_error(trainY_original, trainPredict))
testScore = np.sqrt(mean_squared_error(testY_original, testPredict))
print('Train Score: %.2f RMSE' % trainScore)
print('Test Score: %.2f RMSE' % testScore)

In [ ]:
# Extract date index for plotting
dates = abbv_data.index[look_back+1:look_back+1+len(trainY_original)+len(testY_original)]

The original and predicted prices are plotted against time for both training and test datasets. 

In [ ]:
# Plotting baseline and predictions
plt.figure(figsize=(12, 6))
plt.plot(dates[:len(trainY_original)], trainY_original, label='Original Train')
plt.plot(dates[:len(trainY_original)], trainPredict, label='Predicted Train')
plt.plot(dates[len(trainY_original):], testY_original, label='Original Test')
plt.plot(dates[len(trainY_original):], testPredict, label='Predicted Test')
plt.title('Stock Price Prediction')
plt.xlabel('Time (Year)')
plt.ylabel('Stock Price')
plt.legend()
plt.xticks(rotation=45)  # Rotate date labels for better visibility
plt.show()

In [ ]:
def predict_next_month(model, last_data, scaler, look_back=60, days_in_future=30):
    input_data = last_data[-look_back:].reshape(1, look_back, 4)
    future_predictions = []

    for _ in range(days_in_future):
        prediction = model.predict(input_data)
        future_predictions.append(prediction[0,0])
        new_day = np.append(input_data[0, -1, 1:], prediction).reshape(1, 4)
        input_data = np.append(input_data[:, 1:, :], [new_day], axis=1)

    future_predictions = np.array(future_predictions).reshape(-1, 1)
    future_predictions = scaler.inverse_transform(np.c_[future_predictions, np.zeros((len(future_predictions), 3))])[:, 0]

    return future_predictions

# Last 'look_back' days data for input
last_look_back_data = data_scaled[-look_back:]

# Predict the next month
next_month_predictions = predict_next_month(model, last_look_back_data, scaler, look_back, 30)

# Dates for plotting
dates = pd.date_range(start=abbv_data.index[-1], periods=31, freq='D')[1:]  # starts the day after the last date in the dataset

# Plotting the predictions
plt.figure(figsize=(12, 6))
plt.plot(dates, next_month_predictions, label='Predicted Stock Prices', marker='o')
plt.title('Future Stock Price Prediction for the Next 30 Days')
plt.xlabel('Date')
plt.ylabel('Stock Price')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.show()
